In [1]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git


  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-ggdk5z_f
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-ggdk5z_f
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=59b9cd1f1377572cff5bcb460e8e2d94fe8335210f22d56cb6a5cbd500c94973
  Stored in directory: /tmp/pip-ephem-wheel-cache-m5_2xao5/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [2]:
#load extension into environment
%load_ext nvcc_plugin


created output directory at /content/src
Out bin /content/result.out


In [7]:
%%cu
//Addition of vectors
#include<stdlib.h>
#include<stdio.h>
#include<time.h>

void init_array(double *a, const int N);
void print_array(double *a,const int N);
__global__
void vecAdd(double*a,double*b,double*c,int n)
{
 //get thread id
 int id = blockIdx.x*blockDim.x + threadIdx.x;
 
 if(id<n)
   c[id]=a[id]+b[id];

}


int main()
{
 srand(time(NULL));
 int n = 100;
 
 double *a,*b,*c;
 double *d_a,*d_b,*d_c;

 const int size = n*sizeof(double);

 a = (double*)malloc(size);
 b = (double*)malloc(size);
 c = (double*)malloc(size);

 cudaMalloc(&d_a,size);
 cudaMalloc(&d_b,size);
 cudaMalloc(&d_c,size);

 init_array(a,n);
 init_array(b,n);

 print_array(a,n);
 print_array(b,n);

 cudaMemcpy(d_a,a,size,cudaMemcpyHostToDevice);
 cudaMemcpy(d_b,b,size,cudaMemcpyHostToDevice);

 int blockSize,gridSize;
 
 //no. of threads in each thread block
 blockSize = 1024;
 //no of blocks in grid
 gridSize = (int)ceil((float)n/blockSize);

 vecAdd<<<gridSize,blockSize>>>(d_a,d_b,d_c,n);
 
 cudaThreadSynchronize();

 cudaMemcpy(c,d_c,size,cudaMemcpyDeviceToHost);

 print_array(c,n);

 cudaFree(d_a);
 cudaFree(d_b);
 cudaFree(d_c);
 delete[] a;
 delete[] b;
 delete[] c;

 return 0;
}

void init_array(double*a,const int N)
{
  for(int i=0;i<N;i++)
     a[i] = rand()%N + 1;


}

void print_array(double*a,const int N)
{
  for(int i=0;i<N;i++)
    printf("%f ",a[i]);
printf("\n");
}

65.000000 88.000000 85.000000 57.000000 20.000000 65.000000 39.000000 61.000000 67.000000 78.000000 57.000000 65.000000 98.000000 56.000000 19.000000 99.000000 24.000000 38.000000 99.000000 9.000000 64.000000 88.000000 45.000000 12.000000 28.000000 40.000000 66.000000 86.000000 76.000000 86.000000 16.000000 93.000000 73.000000 53.000000 49.000000 44.000000 17.000000 40.000000 56.000000 36.000000 17.000000 12.000000 100.000000 66.000000 68.000000 19.000000 16.000000 43.000000 8.000000 15.000000 51.000000 23.000000 54.000000 47.000000 34.000000 81.000000 38.000000 100.000000 18.000000 13.000000 37.000000 34.000000 57.000000 61.000000 86.000000 58.000000 4.000000 54.000000 97.000000 59.000000 89.000000 65.000000 71.000000 41.000000 30.000000 90.000000 11.000000 97.000000 84.000000 70.000000 63.000000 34.000000 92.000000 16.000000 32.000000 26.000000 49.000000 69.000000 77.000000 66.000000 33.000000 13.000000 51.000000 90.000000 73.000000 36.000000 99.000000 28.000000 42.000000 47.000000 


In [8]:
%%cu
//Matrix matrix multiplication
#include<stdlib.h>
#include<stdio.h>
#include<time.h>

#define BLOCK_SIZE 16
void init_mat(float*a ,const int N,const int M);
void print_mat(float*a ,const int N,const int M);

__global__
void matrixMultiply(float*a,float*b,float*c,int m,int n,int k)
{
 int row = blockIdx.y*blockDim.y + threadIdx.y;
  int col = blockIdx.x*blockDim.x + threadIdx.x;
 int  sum=0;

 if(col<k && row<m)
 {
    for(int i=0;i<n;i++)
     {
        sum += a[row*n+i]*b[i*k+col];
     }
    c[row*k + col]=sum;

 }


}

int main()
{
srand(time(NULL));

float *a,*b,*c;
float *d_a,*d_b,*d_c;

int M=5;
int N=4;
int P =3;
//a = [5*4],b= [4*3], c=[5*3]

a = (float*)malloc(sizeof(float)*M*N);
b = (float*)malloc(sizeof(float)*N*P);
c = (float*)malloc(sizeof(float)*M*P);


init_mat(a,M,N);
init_mat(b,N,P);
//init_mat(c,M,P);


printf("Initial data:\n");
print_mat(a,M,N);
print_mat(b,N,P);
print_mat(c,M,P);

cudaMalloc(&d_a,sizeof(float)*M*N);
cudaMalloc(&d_b,sizeof(float)*N*P);
cudaMalloc(&d_c,sizeof(float)*M*P);

cudaMemcpy(d_a,a,sizeof(float)*M*N,cudaMemcpyHostToDevice);
cudaMemcpy(d_b,b,sizeof(float)*N*P,cudaMemcpyHostToDevice);

//dim3 dimGrid((P + BLOCK_SIZE - 1) / BLOCK_SIZE, (M + BLOCK_SIZE - 1) / BLOCK_SIZE);
dim3 dimGrid(1,1);
dim3 dimBlock(16, 16);
matrixMultiply<<<dimGrid,dimBlock>>>(d_a,d_b,d_c,M,N,P);
cudaMemcpy(c,d_c,sizeof(float)*M*P,cudaMemcpyDeviceToHost);

printf("Final data:\n");
print_mat(c,M,P);

cudaFree(d_a);
cudaFree(d_b);
cudaFree(d_c);

delete[] a;
delete[] b;
delete[] c;

return 0;
}

void init_mat(float*a ,const int N,const int M)
{
   for(int i=0;i<N;i++)
     for(int j=0;j<M;j++)
        a[i*M + j]= rand()%N +1;
}

void print_mat(float*a ,const int N,const int M)
{
   for(int i=0;i<N;i++)
   {
     for(int j=0;j<M;j++)
           {
             printf("%  f",a[i*M + j]);
             }
     printf("\n");
}
printf("\n");
}

Initial data:
 5.000000 4.000000 3.000000 4.000000
 2.000000 5.000000 5.000000 1.000000
 4.000000 1.000000 1.000000 3.000000
 5.000000 2.000000 4.000000 4.000000
 4.000000 3.000000 3.000000 5.000000

 4.000000 3.000000 1.000000
 2.000000 3.000000 3.000000
 1.000000 3.000000 2.000000
 3.000000 3.000000 2.000000

 0.000000 0.000000 0.000000
 0.000000 0.000000 0.000000
 0.000000 0.000000 0.000000
 0.000000 0.000000 0.000000
 0.000000 0.000000 0.000000

Final data:
 43.000000 48.000000 31.000000
 26.000000 39.000000 29.000000
 28.000000 27.000000 15.000000
 40.000000 45.000000 27.000000
 40.000000 45.000000 29.000000




In [9]:
%%cu
//Vector matrix multiplication
#include<stdlib.h>
#include<stdio.h>
#include<time.h>

void init_array(float*a ,const int N);
void init_mat(float*a ,const int N,const int M);
void print_array(float*a ,const int N);
void print_mat(float*a ,const int N,const int M);


__global__
void kernel(float* vec,float* mat,float* out,const int N,const int M)
{
  int tid = threadIdx.x + blockIdx.x*blockDim.x;
  float sum=0;
  if(tid<M)
  {
    if(tid<M)
       {
          for(int i=0;i<N;i++)
             sum = sum + vec[i]*mat[(i*M)+ tid];
          out[tid]=sum;
       }

  }


}

int main()
{
srand(time(NULL));
 
float *a,*b,*c;
float *d_a,*d_b,*d_c;

int N=3;
int M=4;
//a = [1*3],b= [3*4], c=[1*4]

a = (float*)malloc(sizeof(float)*N);
b = (float*)malloc(sizeof(float)*N*M);
c = (float*)malloc(sizeof(float)*M);

init_array(a,N);
init_mat(b,N,M);
init_array(c,M);

printf("Initial data:\n");
print_array(a,N);

printf("\n\n\n\n");
print_mat(b,N,M);
printf("\n\n\n\n");

print_array(c,M);

printf("\n\n\n\n");

cudaMalloc(&d_a,sizeof(float)*N);
cudaMalloc(&d_b,sizeof(float)*N*M);
cudaMalloc(&d_c,sizeof(float)*M);

cudaMemcpy(d_a,a,sizeof(float)*N,cudaMemcpyHostToDevice);
cudaMemcpy(d_b,b,sizeof(float)*N*M,cudaMemcpyHostToDevice);

kernel<<<M/256+1,256>>>(d_a,d_b,d_c,N,M);

cudaError_t err = cudaGetLastError();
if (err != cudaSuccess) 
    printf("Error: %s\n", cudaGetErrorString(err));

cudaMemcpy(c,d_c,sizeof(float)*M,cudaMemcpyDeviceToHost);

printf("Final data:\n");
print_array(c,M);

cudaFree(d_a);
cudaFree(d_b);
cudaFree(d_c);

delete[] a;
delete[] b;
delete[] c;

return 0;
}

void init_array(float* a,const int N)
{
  for(int i=0;i<N;i++)
    a[i]=rand()%N+1;
}

void init_mat(float*a ,const int N,const int M)
{
   for(int i=0;i<N;i++)
     for(int j=0;j<M;j++)
        a[i*M + j]= rand()%N +1;
}

void print_array(float* a,const int N)
{
  for(int i=0;i<N;i++)
     printf("%f  ",a[i]);
printf("\n");
}

void print_mat(float*a ,const int N,const int M)
{
   for(int i=0;i<N;i++)
   {
     for(int j=0;j<M;j++)
           {
             printf("%  f",a[i*M + j]);
             }
     printf("\n");
}
printf("\n");
}


Initial data:
1.000000  1.000000  1.000000  




 1.000000 3.000000 1.000000 1.000000
 2.000000 1.000000 2.000000 2.000000
 2.000000 2.000000 3.000000 1.000000





4.000000  3.000000  4.000000  1.000000  




Final data:
5.000000  6.000000  6.000000  4.000000  

